<a href="https://colab.research.google.com/github/HARINI-RadheyKrishna/Machine_Learning/blob/master/EE599_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW3 - EE599 Systems for Machine Learning, Fall 2023
University of Southern California

Instructors: Arash Saifhashemi, Murali Annavaram

In this homework assignment, we will ask you to use various methods to implement convolution operation, and then measure and analyze the performance of each method.

## Prepare your Google Drive
- Download `ML_Systems_HW3` zip file from GitHub and unzip the it (you may need to rename the unzipped folder).
- Upload unzipped folder to ``My Drive`` in Google Drive.

In [135]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ML_Systems_HW3/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Verify that you are in the correct working directory.

In [136]:
!pwd

/content/drive/MyDrive/ML_Systems_HW3/src


## Create a folder named `build` under `ML_Systems_HW3/src`, which will be used to store executable files.

In [137]:
!mkdir build

mkdir: cannot create directory ‘build’: File exists


In [141]:
!bash compile.csh

Compiled q1_conv2d_naive.cc successfully!
Compiled q2_conv2d_toeplitz.cc successfully!
Compiled q3_conv2d_toeplitz_transB.cc successfully!
Compiled q4_conv2d_toeplitz_avx.cc successfully!
Compiled q5_conv2d_toeplitz_avx_openmp.cc successfully!
Compiled q6_conv2d_toeplitz_blas.cc successfully!
All files compiled successfully!


In [142]:
!bash run.csh


Running q1_conv2d_naive ...
Virtual Memory (KB): 16716
temp: 0
Time taken by function: 19063 milliseconds
Sum of all outputs: 47243571200
-------------------------------------------
Running q2_conv2d_toeplitz ...
Virtual Memory (KB): 79964
temp: 0
Time taken by function: 32989 milliseconds
Sum of all outputs: 47243571200
-------------------------------------------
Running q3_conv2d_toeplitz_transB ...
Virtual Memory (KB): 79964
temp: 0
Time taken by function: 13526 milliseconds
Sum of all outputs: 47243571200
-------------------------------------------
Running q4_conv2d_toeplitz_avx ...
Virtual Memory (KB): 79964
temp: 0
Time taken by function: 35896 milliseconds
Sum of all outputs: 47243571200
-------------------------------------------
Running q5_conv2d_toeplitz_avx_openmp ...
Virtual Memory (KB): 79964
temp: 0
Time taken by function: 167703 milliseconds
Sum of all outputs: 0
-------------------------------------------
Running q6_conv2d_toeplitz_blas ...
Virtual Memory (KB): 79964
te

## Compile and run your code:
* To compile your C++ code, run command `!bash compile.csh`
* To run your executable code, run command `!bash run.csh`
* Check those csh files and modify accordingly when testing your code.
* You can write and test your code on your local machine but make sure you can compile and run them on Colab. Also you should report the performance measured from Colab enverionment.

## Write code and answer all questions in this notebook.
Note that to measure the performance of each code, we want you to flush your cache. All the template files provide code that flush caches, but we need you to find the cache size of your system. Use the command below to display information about your CPU.

In [ ]:
!lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:          6
    Model:               79
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            0
    BogoMIPS:            4399.99
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscal
                         l nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopo
                         logy nonstop_tsc cpuid tsc_known_freq pni pclmulqdq sss
                         e3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes 
                         xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowpref

The cache sizes are:

    L1d:                   32 KiB (1 instance)
    L1i:                   32 KiB (1 instance)
    L2:                    256 KiB (1 instance)
    L3:                    55 MiB (1 instance)

## Q1
Implement 2D Convolution in the ``q1_conv2d_naive.cc`` using nested for loops. Assume batch size = 1, no padding and stride = 1. Check `util.h` file and understand what each function does. Use the micro `INDEX_4D_TO_1D` to help convert 4d index to 1d index. Measure and report runtime  (in milliseconds) and memory usage (in KB). Manually calculate memory usage and report. Does your calculation match with the measurement?

Total Memory Usage = (weightSize + inputSize + outputSize) * 4 bytes (for float or integer both)

= (128*128*3*3 + 128*128*128 + 128*126*126) * 4
= 17106944/1024 KB
= 16706 KB (Actual: 16716 KB)

## Q2
Use Img2col algorithm to convert the input matrix and kernel matrix to toeplitz form in file ``q2_conv2d_toeplitz.cc``. Interpret input toeplitz matrix and kernel toeplitz matrix as 2d matrix, and store both of them in row major. Use nested for loops to perform matrix multiplication in `matMul` function and call it in `main` function . There is another micro `INDEX_2D_TO_1D` that helps convert 2d index to 1d index. Measure and report runtime and memory usage. Manually calculate memory usage and report. Does your calculation match with the measurement?

Manual Memory Calculation:

(output_p + output_q + output_c + outputToeplitzRows + outputToeplitzCols + outputToeplitzSize + outputs + inputToeplitzRows + inputToeplitzols + topleitzInput + weightTopleitzRows + weightTopleitzCols + topleitzWeight) * 4

((126 × 2) + (128 × 3) + (126 × 126) + (128 × 126 × 126 × 2) + (128 × 3 × 3) + (128 × 3 × 3 × 126 × 126) + (128 × 128 × 3 × 3)) * 4

= 90074112/1024

= 87963 KB (Observed = 77964 KB)

## Q3
For `q3_conv2d_toeplitz_tranB.cc`, repeat procedures in Q2, but store kernel toeplitz matrix in column major and modify `matMul` function accordingly. Measure and report the runtime and memory usage.

(output_p + output_q + output_c + output_row + output_col + output + opSize + newinput_h + newinput_w + topinput + newweight_r + newweight_s + topweight) * 4

= (128 + 126 + 126 + 128 + (126 * 126) + (128*126*126*2) + (128*3*3) + (126*126) + (128*3*3*126*126) + (128*3*3) + 128 + (128*3*3*128)) * 4

= 90142224/1024

= 88029.515 KB (Observed 77964 KB)

## Q4
For ``q4_conv2d_toeplitz_avx.cc``, use Intel AVX (Advanced Vector Extensions) instruction set to perform matmul operation. Intel AVX instructions are Single Instruction Multiple Data (SIMD) instructions that can process 8 floating-point operands in a single instruction. Store input toeplitz matrix in row major and kernel toeplitz matrix in column major. An example of using Intel AVX is given in file ``examples/example_vectorsum_simd.cc``. Measure and report runtime and memory usage.

Hint 1: Use `_mm256_add_ps` and `_mm256_mul_ps` instructions.

Hint 2: If a vector is not divisible by 8, the remaining elements in the vector should not be processed by SIMD instruction. Instead, use normal scalar operations.

## Q5

### Part 1
In file ``q5_conv2d_toeplitz_avx_openmp.cc``, futher optimize the AVX matmul operation using ``OpenMP`` library, which enables multi-threaded parallel computing automatically through a simple and flexible interface. An example of using ``OpenMP`` is given in file ``examples/example_vectorsum_simd_omd.cc``. Run the code with different number of threads. Measure and report the runtime for number of threads = [2, 4, 8].


### Part 2 (Optional - No Credit)

In file ``q5_optional_conv2d_toeplitz_avx_multi_thread.cc``, instead of using `OpenMP`, use the C++ standard `<thread>` library to implement multi-threaded AVX matmul operation. You can refer to [this video](https://youtu.be/3aqxaZsvn80?si=1QEE580e2vLmrqPO) to learn about multi threading in C++.


Runtime for 2 threaded core =

Runtime for 4 threaded core =

Runtime for 8 threaded core =

## Q6

In file ``q6_conv2d_toeplitz_blas.cc``, use `BLAS` library to implement matmul operation. You can decide the storage format for input and kernel toeplitz matrices, and make sure you set input arguments of `cblas_sgemm` accordingly. An example is given in file ``examples/example_matmul_blas.cc``. Search online document of `cblas_sgemm` API if you are unclear about the mearning of each of its input argument. Follow the example and and finish your own code. Measure and report runtime and memory usage.

## Q7  

### Part 1


Analyze the performance differences between Q1-Q6. Explain what constitutes the performance difference between each implementations and why `BLAS` library is super fast.

### Part 2: Using Google Benchmark (Optional - No Credit)
As an optional practice, you can measure the runtime of each matrix calculation method more accurately using Google benchmark.

A better way to measure performance of a funcion in C++ is to use Google Benchmark. You can refer to [this video](https://youtu.be/9VKR8u9odrA?si=xSInuzT5uMBOKAbP) to familiarize yourself with this package.




## Upload files to GitHub
Make sure upload your final C++ code and this IPython notebook to GitHub Repo either mannully or through git commands.